In [46]:
import numpy as np
import pennylane as q
import multiprocessing as mp
import matplotlib.pyplot as plt
from original_grover_noise import original_grover_iter
from exact_grover_noise import modified_grover_iter

In [52]:
def DEGA(target_combo, p):
    n_bits = len(target_combo)
    all_wires = list(range(n_bits))
    dev = q.device("default.mixed", wires=n_bits, shots=1000, seed=42)

    @q.qnode(dev)
    def inner_circuit():
        if n_bits%2:
            if len(target_combo) != 3:
                for i in range((len(target_combo)-3)//2):
                    original_grover_iter([int(target_combo[i*2]), int(target_combo[2*i+1])], p, list(range(2*i,2*i+2)))
            modified_grover_iter([int(target_combo[-3]), int(target_combo[-2]), int(target_combo[-1])], p, [n_bits-3,n_bits-2,n_bits-1])
        else:
            for i in range(len(target_combo)//2):
                original_grover_iter([int(target_combo[i*2]), int(target_combo[2*i+1])], p, list(range(2*i,2*i+2)))
    
        return q.probs(wires=all_wires)
    return inner_circuit()

def get_plot(combo, values):
    n=len(combo)
    categories = np.vectorize(lambda x: np.binary_repr(x, width=n))(np.arange(2**n)).tolist()
    plt.bar(categories, values, color='skyblue')
    plt.xticks(rotation=90)
    plt.title(f'{len(combo)}-qubit, target={combo}')
    plt.ylabel('Probability')
    plt.show()

In [64]:
# get_plot('11', DEGA([0,0,1,1], 0.03)[1])
noise_param=np.arange(0, 0.091, 0.01)
dega_res=[]
for p in noise_param:
    dega_res.append(DEGA([0,0,1,1,0], p)[0])